In [2]:
import pandas as pd
import telegram
from datetime import datetime as dt
from binance.client import Client
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from sqlalchemy import create_engine
from secrets import secrets

In [3]:
TELEGRAM_CHAT_ID = secrets['TELEGRAM_CHAT_ID']
TELEGRAM_TOKEN = secrets['TELEGRAM_TOKEN']
BINANCE_API_KEY = secrets['BINANCE_API_KEY']
BINANCE_SECRET_KEY = secrets['BINANCE_SECRET_KEY']
DATABASE_URL = secrets['DATABASE_URL']
BRIDGE = 'USDT'
TRANS_FEE = 0.0075

In [3]:
df_invest = pd.DataFrame(columns=['Id', 'Date', 'Amount'])
df_buy = pd.DataFrame(columns=['Id', 'Date', 'Coin', 'UsdAmount', 'CoinAmount'])
df_sell = pd.DataFrame(columns=['Id', 'Date', 'Coin', 'CoinAmount', 'UsdAmount'])

In [4]:
# df_pf = pd.DataFrame(columns=['Id', 'Coin', 'Amount'])
df_trans = pd.DataFrame(columns=['Id', 'Date', 'Action', 'USD', 'Coin', 'Amount', 'Price'])

### Invest

In [5]:
def modify_portfolio(chat_id, coin, amount):
    curr_amount = df_pf.loc[(df_pf['Id'] == chat_id) & (df_pf['Coin'] == coin), 'Amount']
    if len(curr_amount) > 0:
        df_pf.loc[(df_pf['Id'] == chat_id) & (df_pf['Coin'] == coin), 'Amount'] += amount
    else:
        df_pf.loc[len(df_pf)] = [chat_id, coin, amount]

In [6]:
def invest(chat_id, amount):
    df_trans.loc[len(df_trans)] = [chat_id, dt.now(), 'IN', amount, None, None, None]
    # modify_portfolio(chat_id, 'USD', amount)

### Buy

In [7]:
def check_price(coin):
    symbol = coin + BRIDGE
    client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
    t = client.get_ticker(symbol=symbol)
    return float(t['lastPrice'])

In [8]:
def buy(chat_id, coin, amount):
    df_temp = df_trans.loc[df_trans['Id'] == chat_id]
    curr_amount = df_trans['USD'].sum()
    if curr_amount < amount:
        return -1
    price = check_price(coin)
    new_amount = (amount / price) * (1 - TRANS_FEE)
    df_trans.loc[len(df_trans)] = [chat_id, dt.now(), 'BUY', -1 * amount, coin, new_amount, price]
    # df_pf.loc[(df_pf['Id'] == chat_id) & (df_pf['Coin'] == 'USD'), 'Amount'] -= amount
    # modify_portfolio(chat_id, coin, new_amount)
    return 0

### Sell

In [9]:
def sell(chat_id, coin, amount):
    df_temp = df_trans.loc[(df_trans['Id'] == chat_id) & (df_trans['Coin'] == coin)]
    curr_amount = df_trans['Amount'].sum()
    if curr_amount < amount:
        return -1
    price = check_price(coin)
    new_amount = amount * price * (1 - TRANS_FEE)
    df_trans.loc[len(df_trans)] = [chat_id, dt.now(), 'SELL', new_amount, coin, -1 * amount, price]
    # df_pf.loc[(df_pf['Id'] == chat_id) & (df_pf['Coin'] == coin), 'Amount'] -= amount
    # modify_portfolio(chat_id, 'USD', new_amount)
    return 0

### Test

In [ ]:
invest(CHAT_ID, 1000)
buy(CHAT_ID, 'BTC', 1000)
sell(CHAT_ID, 'BTC', 0.01)
buy(CHAT_ID, 'BNB', 500)
sell(CHAT_ID, 'BNB', 0.5)

In [ ]:
df_trans

In [ ]:
df_trans.info()

### Portfolio

In [ ]:
def portfolio(chat_id):
    df_temp = df_trans.loc[df_trans['Id'] == chat_id]
    initial = df_temp.loc[df_temp['Action'] == 'IN', 'USD'].sum()
    curr_usd = df_temp['USD'].sum()
    df_coins = df_temp.groupby('Coin')['Amount'].sum().reset_index()
    df_coins = df_temp.groupby('Coin')['Amount'].sum().reset_index()
    df_coins['Price'] = df_coins['Coin'].apply(lambda x: check_price(x))
    df_coins['Value'] = df_coins['Amount'] * df_coins['Price']
    curr_val = curr_usd + df_coins['Value'].sum()
    profit = curr_val - initial
    percent = profit / initial
    return curr_val, profit, percent

In [ ]:
portfolio(CHAT_ID)

In [ ]:
df_coins = df_trans.groupby('Coin')['Amount'].sum().reset_index()
df_coins['Price'] = df_coins['Coin'].apply(lambda x: check_price(x))
df_coins['Value'] = df_coins['Amount'] * df_coins['Price']
df_coins.head()

### To SQL

In [ ]:
engine = create_engine(DATABASE_URL)
# df_pf.to_sql('Portfolio', engine, index=False, if_exists='replace')
df_trans.to_sql('Transaction', engine, index=False, if_exists='replace')
engine.dispose()

## Telegram

In [25]:
engine = create_engine(DATABASE_URL)
user_id = 123
df = pd.read_sql_query('select * from "Transaction" where "Id" = {}'.format(user_id), engine)
df

,Id,Date,Action,USD,Coin,Amount,Price
0,123,2021-04-12 00:00:31.111637,IN,1000.000000,None,NaN,NaN
1,123,2021-04-12 00:00:31.966630,BUY,-1000.000000,BTC,0.016608,59758.7600
2,123,2021-04-12 00:00:32.785623,SELL,593.108373,BTC,-0.010000,59759.0300
3,123,2021-04-12 00:00:33.786456,BUY,-500.000000,BNB,1.004580,493.9876
4,123,2021-04-12 00:00:34.569564,SELL,245.147500,BNB,-0.500000,494.0000


In [26]:
usd = df['USD'].sum()
df = df.groupby('Coin')['Amount'].sum().reset_index()
df.loc[len(df)] = ['USD', usd]

In [28]:
df.loc[df['Coin'] == 'USD', 'Amount'].item()

338.25587275

In [56]:
def get_holdings(user_id, engine):
    df = pd.read_sql_query('select * from "Transaction" where "Id" = {}'.format(user_id), engine)
    usd = df['USD'].sum()
    df = df.groupby('Coin')['Amount'].sum().reset_index()
    df.loc[len(df)] = ['USD', usd]
    return df.to_dict('records')

def invest_func(user_id, amount):
    try:
        df = pd.DataFrame(columns=cols)
        df.loc[len(df)] = [user_id, dt.now(), 'IN', amount, None, None, None]
        engine = create_engine(DATABASE_URL)
        add_transaction(user_id, 'IN', amount, None, None, None, engine)
        engine.dispose()
        return 0
    except:
        return -1

def buy_func(user_id, coin, amount, price):
    engine = create_engine(DATABASE_URL)
    holdings = get_holdings(user_id, engine)
    curr_amount = holdings['USD']
    if curr_amount < amount:
        return curr_amount - amount
    new_amount = (amount / price) * (1 - TRANS_FEE)
    add_transaction(user_id, 'BUY', -1 * amount, coin, new_amount, price, engine)
    engine.dispose()
    return new_amount

def sell_func(user_id, coin, amount, price):
    engine = create_engine(DATABASE_URL)
    holdings = get_holdings(user_id, engine)
    curr_amount = holdings[coin]
    if curr_amount < amount:
        return curr_amount - amount
    new_amount = amount * price * (1 - TRANS_FEE)
    add_transaction(user_id, 'SELL', new_amount, coin, -1 * amount, price, engine)
    engine.dispose()
    return new_amount

def portfolio_func(user_id):
    engine = create_engine(DATABASE_URL)
    df = pd.read_sql_query('select * from "Transaction" where "Id" = {}'.format(user_id), engine)
    if len(df) < 1:
        return -1, -1
    engine.dispose()
    initial = df.loc[df['Action'] == 'IN', 'USD'].sum()
    curr_usd = df['USD'].sum()
    df = df.groupby('Coin')['Amount'].sum().reset_index()
    df['Price'] = df['Coin'].apply(lambda x: check_price(x))
    df['Value'] = df['Amount'] * df['Price']
    df.loc[len(df)] = ['USD', curr_usd, 1, curr_usd]
    return initial, df

def add_transaction(user_id, action, usd, coin, amount, price, engine):
    df = pd.DataFrame(columns=['Id', 'Date', 'Action', 'USD', 'Coin', 'Amount', 'Price'])
    df.loc[len(df)] = [user_id, dt.now(), action, usd, coin, amount, price]
    df.to_sql('Transaction', engine, index=False, if_exists='append')

def check_price(coin):
    symbol = coin + BRIDGE
    client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
    t = client.get_ticker(symbol=symbol)
    return float(t['lastPrice'])

In [57]:
curr_val = df['Value'].sum()
profit = curr_val - i
percent = profit / i * 100
coins = df.to_dict('records')

template = """Portfolio:\t ${:.4f}
Initial:\t ${:.2f}
Profit:\t\t ${:.4f} ({:.2f}%)
Wallet:""".format(curr_val, i, profit, percent)

for coin in coins:
    template += "\n- {:.4f} {}\t ~ ${:.2f}".format(coin['Amount'], coin['Coin'], coin['Value'])

print(template)

Portfolio:	 $993.8629
Initial:	 $1000.00
Profit:		 $-6.1371 (-0.61%)
Wallet:
- 1.0381 BNB	 ~ $596.78
- 0.1067 ETH	 ~ $227.75
- 169.3340 USD	 ~ $169.33


In [23]:
def invest(update, context):
    try:
        amount = float(context.args[0])
        print(amount)
    except:
        update.message.reply_text(text='Please provide a valid amount to invest.')
        return
    user_id = update.message.chat.id
    res = invest_func(user_id, amount)
    if res == 0:
        update.message.reply_text(text='Successfully invested ${}.'.format(amount))
    else:
        update.message.reply_text(text='Error encountered. Please try again later.')

def buy(update, context):
    if len(context.args) != 2:
        update.message.reply_text(text='Please exactly two inputs, COIN and AMOUNT.')
        return
    try:
        coin = context.args[0].upper()
        price = check_price(coin)
    except:
        update.message.reply_text(text='Please provide a valid coin.')
        return
    try:
        amount = float(context.args[1])
    except:
        update.message.reply_text(text='Please provide a valid amount to buy.')
        return
    user_id = update.message.chat.id
    res = buy_func(user_id, coin, amount, price)
    if res >= 0:
        update.message.reply_text(text='Successfully bought {} {} (Priced @ {}) for ${}.'\
                                  .format(res, coin, price, amount))
    else:
        update.message.reply_text(text='Not enough USD for transaction. Missing ${}.'.format(-1 * res))

def sell(update, context):
    if len(context.args) != 2:
        update.message.reply_text(text='Please exactly two inputs, COIN and AMOUNT.')
        return
    try:
        coin = context.args[0].upper()
        price = check_price(coin)
    except:
        update.message.reply_text(text='Please provide a valid coin.')
        return
    try:
        amount = float(context.args[1])
    except:
        update.message.reply_text(text='Please provide a valid amount to sell.')
        return
    user_id = update.message.chat.id
    res = sell_func(user_id, coin, amount, price)
    if res >= 0:
        update.message.reply_text(text='Successfully sold {} {} (Priced @ {}) for ${}.'\
                                  .format(amount, coin, price, res))
    else:
        update.message.reply_text(text='Not enough {} for transaction. Missing {}.'.format(coin, -1 * res))

def portfolio(update, context):
    user_id = update.message.chat.id
    initial, df = portfolio_func(user_id)
    if initial == -1:
        update.message.reply_text(text='Portfolio not found.')
    else:
        curr_val = df['Value'].sum()
        profit = curr_val - initial
        percent = profit / i * 100
        coins = df.to_dict('records')
        # Message
        portfolio_temp = """Portfolio:\t ${:.4f}\nInitial:\t ${:.2f}\nProfit:\t\t ${:.4f} ({:.2f}%)\nWallet:"""\
            .format(curr_val, i, profit, percent)
        for coin in coins:
            portfolio_temp += "\n- {:.4f} {}\t ~ ${:.2f}".format(coin['Amount'], coin['Coin'], coin['Value'])
        update.message.reply_text(text=portfolio_temp)

def main():
    updater = Updater(token=TELEGRAM_TOKEN)
    # Bot commands
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('invest', invest))
    dp.add_handler(CommandHandler('buy', buy))
    dp.add_handler(CommandHandler('sell', sell))
    # Job queue
    job_queue = updater.job_queue
    updater.start_polling()
    updater.idle()

In [24]:
main()

KeyboardInterrupt: 